#**Intermediate Bioinformatics Exercise** — Single-cell transcriptomics


Today, we will go through a standard analysis workflow for single-cell RNA-sequencing (scRNA-seq) data using [scanpy](https://scanpy.readthedocs.io/en/stable/index.html), a powerful and scalable Python library for single-cell analysis.  
<br>

You'll learn how to:
* Work with [AnnData](https://anndata.readthedocs.io/en/stable/) objects
* Load and explore scRNA-seq datasets in Python
* Perform standard preprocessing steps (e.g., filtering cells/genes, normalization, log transformation, dimensionality reduction)
* Compute and visualize neighborhood graphs and UMAP embeddings
* Apply clustering algorithms and annotate cell populations


<!-- *   Quality control
*   Normalization
*   Dimensionality reduction (feature selection and PCA)
*   UMAP visualization
*   Clustering
*   Cell-type annotation -->

<br>

An expanded version of this tutorial is available [here](https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html).

---

# Imports

In [ ]:
# Core scverse libraries
!pip install scanpy
!pip install anndata
!pip install pooch
!pip install igraph
!pip install gseapy

import scanpy as sc
import anndata as ad
import pooch
import pandas as pd
from gseapy.parser import download_library

# plotting parameters
sc.settings.set_figure_params(dpi=150, facecolor="white")

sc.settings.verbosity = 2

import warnings
warnings.filterwarnings("ignore", message=".*names are not unique.*")

# Load in the data

The data used in this tutorial was collected from bone marrow mononuclear cells of healthy human donors from [Luecken et al. 2021](https://datasets-benchmarks-proceedings.neurips.cc/paper_files/paper/2021/file/158f3069a435b314a80bdcb024f8e422-Paper-round2.pdf).


We are reading in the count matrix into an [AnnData](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html) object, which holds many slots for annotations and different representations of the data.

In [ ]:
EXAMPLE_DATA = pooch.create(
    path=pooch.os_cache("scverse_tutorials"),
    base_url="doi:10.6084/m9.figshare.22716739.v1/",
)
EXAMPLE_DATA.load_registry_from_doi()



# We are loading in 2 samples, each from a different donor
samples = {
    "s1d1": "s1d1_filtered_feature_bc_matrix.h5",
    "s1d3": "s1d3_filtered_feature_bc_matrix.h5",
}
adatas = {}

for sample_id, filename in samples.items():
    path = EXAMPLE_DATA.fetch(filename)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique() # ensures cell barcodes are unique

print("\nFinished loading samples.\n")
adata

In [ ]:
print(f"Total number of cells: {adata.n_obs}\n")
print(f"Cells per sample:\n{adata.obs['sample'].value_counts()}\n")

adata.obs.head()

In [ ]:
print(f"Total number of genes: {adata.n_vars}\n")

adata.var.head()

# Preprocessing

The following code block groups together the standard preprocessing steps for scRNA-seq data, including:
* Quality control
* Cell and gene filtering
* Data normalization
* Dimensionality reduction (feature selection and PCA)

Each step typically requires optimization depending on the specific dataset. For the purposes of today's breakout session, we perform all steps below and move on to data visualization and cell type annotation.

Example plots to evaluate preprocessing parameters are included if you wish to explore further.

In [ ]:
### ----- Quality Control -----

adata.var["mt"] = adata.var_names.str.startswith("MT-") # mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL")) # ribosomal genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]") # hemoglobin genes

sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

# # QC plot example
# sc.pl.violin(
#     adata,
#     ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
#     jitter=0.4,
#     multi_panel=True,
# )



### ----- Cell and Gene Filtering -----

sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)



### ----- Normalization -----

# Saving raw count data
adata.layers["counts"] = adata.X.copy()

# Normalizing to median total counts
sc.pp.normalize_total(adata)

# Logarithmize the data
sc.pp.log1p(adata)

# Save log-normalized counts
adata.layers['log_norm'] = adata.X.copy()



### ----- Dimensionality Reduction -----

# Feature selection
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample")

# sc.pl.highly_variable_genes(adata)

# PCA
sc.tl.pca(adata)

# sc.pl.pca(
#     adata,
#     color=["sample", "sample", "pct_counts_mt", "pct_counts_mt"],
#     dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
#     ncols=2,
#     size=2,
# )


# Inspect new columns added to adata.obs (cell-level) and adata.var (gene-level)
print(f"\n{adata}")

# Nearest neighbor graph and UMAP

Now that we have preprocessed our dataset, we will compute nearest neighbors using the PCA representation of the data and embed the neighbors graph in 2 dimensions with UMAP.

In [ ]:
### ----- UMAP Visualization -----

# Compute nearest neighbors
sc.pp.neighbors(adata)

# Embed neighbors graph in 2D with UMAP
sc.tl.umap(adata)

# Plot UMAP
sc.settings.set_figure_params(figsize=(5, 5))

sc.pl.umap(
    adata,
    color="sample", # can use any .obs or .var column
    size=5,
)

# Clustering

After visualizing cells in a UMAP plot, the next step is **clustering**, which groups cells based on their gene expression profiles. Cells with similar gene expression will be clustered together, helping to identify distinct cell types or states within a mixed population. These clusters can then be annotated to assign likely cell types.

<br>

Here, we use the **Leiden graph-clustering method** (community detection based on optimizing modularity) which directly clusters the neighbor graph of cells that we computed in the previous section. The `resolution` parameter controls the coarseness of clustering, where higher values result in more clusters. We start with a low resultion to explore broad cell types.


In [ ]:
sc.tl.leiden(
    adata,
    flavor="igraph",
    n_iterations=5,
    resolution=0.02, # default = 1
)

# Inspect clusters in UMAP
sc.pl.umap(
    adata,
    size=15,
    color=["leiden"],
    palette='tab10',
    legend_loc='on data',
    legend_fontoutline=2,
    title="leiden (res=0.02)",
)

# Cell-Type Annotation

We will now annotate our clusters to known cell types. This is typically done using known marker genes for the cells in our data (i.e., genes that are exclusively expressed by a given cell type).

Several databases/resources exist containing cell-type specific marker genes:
* [CellMarker](http://bio-bigdata.hrbmu.edu.cn/CellMarker/)
* [TF-Marker](http://bio.liclab.net/TF-Marker/)
* [PanglaoDB](https://panglaodb.se/)

Below, we have curated a list of relevent cell types and marker genes for today's breakout session. We will start with broad lineage markers.




In [ ]:
lineage_marker_genes = {
    'Myeloid': ['LYZ', 'CD33', 'CD14'],
    'Lymphoid': ['CD3D', 'CD79A', 'NKG7'],
    'Erythroid': ['HBB', 'GYPA', 'ALAS2'],
}

for lineage, genes in lineage_marker_genes.items():
    # Make sure genes are present in our dataset
    genes_to_plot = [gene for gene in genes if gene in adata.var_names]

    print(f"{lineage} cells")

    sc.pl.umap(
        adata,
        color=genes_to_plot,
        ncols=3,
        size=15,
        layer='log_norm',
    )

There are fairly clear patterns of expression for our lineage markers shown here, which we can use to broadly label our clusters.

In [ ]:
adata.obs["lineage_annotation"] = adata.obs["leiden"].map(
    {
        "0": "Lymphoid",
        "1": "Myeloid",
        "2": "Lymphoid",
        "3": "Erythroid",
        "4": "Erythroid",
    }
)

sc.pl.umap(
    adata,
    color=["leiden", "lineage_annotation"],
    size=15,
    ncols=2,
    palette='tab10',
    legend_loc='on data',
    legend_fontoutline=2,
    title=["leiden (res=0.02)", "lineage_annotation"],
)

At low clustering resolution (e.g., 0.02), cells are grouped into broad lineages, which helps us to identify major cell types using a small set of marker genes. But even within these broad clusters, there are clear differences among cells, suggesting hidden heterogeneity that our current resolution does not fully capture.

In [ ]:
# E.g., the Lymphoid marker genes
sc.pl.umap(
    adata,
    color=['lineage_annotation', 'CD3D', 'CD79A', 'NKG7'],
    size=15,
    ncols=4,
    legend_loc='on data',
    legend_fontoutline=2,
)

<br>

---------------------------------------------------------------

# Hands-On Challenge: Resolving Subpopulations

Next, we'll zoom in and try to resolve this heterogeneity. Below, we provide:
* Marker genes for specific cell types
* Code for adjusting and exploring clustering resolutions
* Additional `scanpy` functions (optional)

<br>

Your **GOAL** is to explore gene expression and adjust the clustering resolution to best separate and annotate these subpopulations of cells.

<br>

*Note: Cell type annotation typically requires multiple rounds of subclustering, re-annotation, and domain knowledge. The exact number of clusters is somewhat arbitrary, and results are never perfect---especially in a short session like this. For now, aim for a clustering that makes sense for annotating our cell types, even if it's not exact.*

In [ ]:
cell_type_markers = {
    ### Myeloid
    'Monocyte': ['CD14', 'FCN1', 'S100A8'],

    ### Lymphoid
    'T cell': ['CD3E', 'IL7R', 'TRAC'],
    'CD8+ T cell': ['CCL5', 'GZMA', 'GZMH'],
    'T naive': ['CCR7', 'LEF1', 'TCF7'],
    'B cell': ['CD79A', 'MS4A1', 'IGHD'],
    'Transitional B': ['MME', 'CD38', 'ACSM3'],
    'Plasma cell': ['JCHAIN', 'MZB1', 'XBP1'],

    ### Erythroid
    'Erythroblast': ['HBA1', 'MKI67', 'GYPA'],
    'Proerythroblast': ['CD34', 'CDK6', 'SYNGR1'],
}

# Note that some markers are expressed in multiple cell types

In [ ]:
resolutions = [0.02, 0.5, 2.0] # Choose clustering resolutions here

for res in resolutions:
    sc.tl.leiden(
        adata,
        key_added=f"leiden_res_{res}", # adds to adata.obs
        resolution=res, # default = 1
        flavor="igraph",
    )

# Visualize clusters
sc.settings.set_figure_params(figsize=(5, 5))
sc.pl.umap(
    adata,
    color=[f"leiden_res_{res}" for res in resolutions],
    size=15,
    palette='tab20',
    legend_loc="on data",
    legend_fontoutline=2,
)

*Hint: Resolution = 2.0 overclusters the data.*

### Additional `scanpy` functions

[All scanpy plotting functions](https://scanpy.readthedocs.io/en/stable/api/plotting.html)

In [ ]:
### ----- Dotplot of gene expression -----

sc.pl.dotplot(
    adata,
    cell_type_markers, # input list of genes
    groupby='leiden',
    standard_scale='var',
)


### ----- Plot highest expressed genes -----

sc.pl.highest_expr_genes(
    adata,
    n_top=10,
    layer='log_norm',
)


### ----- Gene scoring -----

# Score a gene, or set of genes, against a randomly sampled reference set of genes (similar to Z-score)
# Can plot resulting score (adata.obs['score']) on UMAP embedding

gene_list = cell_type_markers['B cell'] # Example: B Cell marker genes from the curated list

sc.tl.score_genes(
    adata,
    gene_list=gene_list, # input list of genes
    ctrl_size=500, # can adjust
    score_name='score', # score name to be added to adata.obs
    layer='log_norm',
)


### ----- Differential Gene Expression (DGE) -----

# Computes marker genes per group/cluster using simple statistical tests; compares each group against the union of the rest
sc.tl.rank_genes_groups(
    adata,
    groupby='leiden', # cell grouping from adata.obs to consider (e.g., our leiden clusters from above)
    method='wilcoxon',
    corr_method='benjamini-hochberg',
    use_raw=False,
    layer='log_norm',
    pts=True, # computes fraction of cells expressing the genes
)

# Extract DGE results as a dataframe
dge = sc.get.rank_genes_groups_df(adata, group=None) # can subset for a specific cluster with group param (e.g., group='3')

# Example plot to visualize DE genes
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby='leiden',
    standard_scale="var",
    n_genes=5, # n genes to show per group
    # values_to_plot='logfoldchanges', # default is mean expression within the group
)

<br>

---------------------------------------------------------------

In [ ]:
more_markers = {
    'Monocyte': ['CD14', 'FCN1', 'S100A8', 'ITGAM', 'CCR2', 'LYN'],
    'T cell': ['CD3E', 'IL7R', 'TRAC', 'CD3G', 'TRA'],
    'CD8+ T cell': ["CD8A", "CD8B", "GZMK", "GZMA", "CCL5", "GZMB", "GZMH", "GZMA"],
    'T naive': ['CCR7', 'LEF1', 'TCF7', 'PTPRC', 'SELL'],
    'B cell': ['CD79A', 'MS4A1', 'IGHD', 'CD19', 'CR2', 'CD79B'],
    'Transitional B': ["MME", "CD38", "CD24", "ACSM3", "MSI2", 'CD27', 'IGHD'], # CD27 is a negative marker
    'Plasma cell': ['JCHAIN', 'MZB1', 'XBP1', "HSP90B1", "FNDC3B", "PRDM1", "IGKC", 'PAX5'], # PAX5 is a negative marker
    'Erythroblast': ['HBA1', 'MKI67', 'GYPA', 'TFRC', 'KMT5A', 'TAL1'],
    'Proerythroblast': ['CD34', 'CDK6', 'SYNGR1', 'HBM', 'GYPA', 'ENG', 'MYB'], # HBM and GYPA are negative markers
}